In [1]:
from transformers import AutoModel, AutoTokenizer

# Replace 'bert-base-cased' with the appropriate BERT model name
model_name = 'model_dir'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

# Replace 'input_text' with your sample input text
input_text = "Sample input text"
inputs = tokenizer(input_text, return_tensors='pt')
with torch.no_grad():
    outputs = model(**inputs)


In [3]:
import onnxruntime

# Replace 'onnx_dir/optimized_bert.onnx' with the path to your optimized BERT ONNX model
onnx_model_path = 'onnx_dir/onnx_bert/bert.onnx'
ort_session = onnxruntime.InferenceSession(onnx_model_path)


In [4]:
import numpy as np

# Convert inputs to ONNX-compatible format (input tensor names can be found in the model's inputs)
input_name = ort_session.get_inputs()[0].name
# # Ensure 'token_type_ids' is present in the input feed
# input_name = ort_session.get_inputs()[0].name
inputs_onnx = {
    input_name: np.array(inputs['input_ids']),
    'attention_mask': np.array(inputs['attention_mask']),
    'token_type_ids': np.array(inputs['token_type_ids'])
}

outputs_onnx = ort_session.run(None, inputs_onnx)



In [5]:
import time

# Measure inference time for the original BERT model
start_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_time = time.time()
original_inference_time = end_time - start_time

# Measure inference time for the optimized BERT model
start_time = time.time()
outputs_onnx = ort_session.run(None, inputs_onnx)
end_time = time.time()
optimized_inference_time = end_time - start_time

# Compare outputs (outputs_onnx and outputs) to ensure they match
print(f"Original model run time:{original_inference_time}")
print(f"Onnx model run time: {optimized_inference_time}")


Original model run time:0.1712489128112793
Onnx model run time: 0.05049324035644531
